In [3]:
#%%writefile Cointegration.py
#https://medium.com/@bart.chr/pairs-trading-for-algorithmic-trading-breakdown-d8b709f59372
#https://github.com/aconstandinou/mean-reversion

import statsmodels.api as sm
import statsmodels.tsa.stattools as ts 
from statsmodels.tsa.stattools import coint
import numpy as np
import pandas as pd
import statistics

import matplotlib.pyplot as plt

# Functions/Methods

In [62]:
"""
Augmented Dickey–Fuller (ADF) unit root test; p-value < .05
"""
class DickeyFuller(object):
    def __init__(self, significance=.05):
        self.significance_level = significance
        self.p_value = None
        self.perc_stat = None
        self.is_stationary = None
        
    def check(self, time_series):
        model = ts.adfuller(time_series, 1)
        self.p_value = model[1]
        statistic = model[0]
        
        # Dickey-Fuller
        self.is_stationary = False
        if (self.p_value < self.significance_level):
            self.is_stationary = True
        
        # Augmented Dickey Fuller (ADF)
        if (abs(statistic) > abs(model[4]['1%'])):
            self.perc_stat = 99
        elif (abs(statistic) > abs(model[4]['5%'])):
            self.perc_stat = 95
        elif (abs(statistic) > abs(model[4]['10%'])):
            self.perc_stat = 90
    
        return self.is_stationary;

"""
Half Life test from the Ornstein-Uhlenbeck process 
"""
class HalfLife(object):
    def __init__(self):
        self.half_life = None

    def check(self, time_series):
        lag = np.roll(time_series, 1)
        lag[0] = 0
        ret = time_series - lag
        ret[0] = 0

        # adds intercept terms to X variable for regression
        lag2 = sm.add_constant(lag)
        res = sm.OLS(ret, lag2).fit()
        self.half_life = int(round(-np.log(2) / res.params[1],0))

        if self.half_life <= 0:
            self.half_life = 1
        return self.half_life

"""
If Hurst Exponent is under the 0.5 value of a random walk, then the series is mean reverting
"""
class HurstExponent():
    def __init__(self):
        self.h_min = 0.0
        self.h_max = 0.4
        self.look_back = 126
        #https://robotwealth.com/demystifying-the-hurst-exponent-part-1/
        self.lag_max = 20#era 100
        self.h_value = None
    
    def check(self, time_series):
        lags = range(2, self.lag_max)

        tau = [np.sqrt(np.std(np.subtract(time_series[lag:], time_series[:-lag]))) for lag in lags]
        poly = np.polyfit(np.log(lags), np.log(tau), 1)

        self.h_value = poly[0]*2.0 
        return self.h_value

def model_ols(y, x):
    x = sm.add_constant(x)
    model = sm.OLS(y, x).fit()
    return model

# beta/coeficiente angular
def beta(y, x):
    model = model_ols(y, x)
    return model.params[1]

# check cointegrated pairs from dataframe
def find_cointegrated_pairs(data, num_pairs=0, period=250):
    adf = DickeyFuller()
    rows = []
    isBreak = False
    index=-1
    
    for y_symbol in data.columns:
        index = index + 1
        for x_symbol in data.columns[index+1:data.shape[1]]:#for x_symbol in data.columns:
            if (y_symbol == x_symbol):
                continue
            
            x = data[x_symbol]
            y = data[y_symbol]
            
            # filter by period
            if (period > 0):
                pos = len(x)-period
                y = y.iloc[pos:]
                x = x.iloc[pos:]
            
            model = model_ols(y, x)
            adf.check(model.resid)
            beta = model.params[1]
            
            # check is stationary
            if (adf.is_stationary):
                rows.append([len(x), y_symbol,x_symbol,adf.p_value, adf.perc_stat, beta])
                    
            # break for two
            isBreak = (num_pairs > 0 and len(rows) >= num_pairs)
            if (isBreak == True): break
        
        # break for one
        if (isBreak == True): break

    df_pairs = pd.DataFrame(rows, columns=['Period', 'Dependente', 'Independente', 'Dickey-Fuller', 'ADF', 'Beta'])
    return df_pairs

def apply_periods(data, pairs):
    pairs['Period'] = 0
    pairs['PeriodStr'] = ''
    for i, row in pairs.iterrows():
        y = data[row['Dependente']]
        x = data[row['Independente']]
        analysis = analysis_by_periods(y, x)
        stationary = analysis.loc[(analysis['Stationary'])]

        des = ''
        for j, row in stationary.iterrows():
            if (des!=''):
                des=des+','
            des=des+str(row['Period'])

        pairs['Period'].iloc[i] = stationary.shape[0]
        pairs['PeriodStr'].iloc[i] = des
        
def analysis_by_periods(y, x):
    rows=[]
    n = len(y)
    adf = DickeyFuller()
    
    for period in [100, 120, 140, 160, 180, 200, 220, 240, 250]:
        pos = n-period
        y_values = y.iloc[pos:]
        x_values = x.iloc[pos:]
        
        coin = cointegration(y_values, x_values, 0)
        half_life = check_halflife(y_values, x_values)
        hurst = check_hurst(y_values, x_values)
        corr = corr_pearson(return_varlog(y_values), return_varlog(x_values))
            
        rows.append([period, coin[0], coin[1], coin[1], coin[2], half_life, hurst, corr])
        
    analysis = pd.DataFrame(rows, columns=['Period', 'Stationary', 'Dickey-Fuller', 'ADF', 'Beta', 'HalfLife', 'Hurst', 'Corr'])
    return analysis

def return_varlog(time_series):
    lag = np.roll(time_series, 1)
    lag[0] = 0
    ret = np.log(time_series/lag)
    ret[0] = 0
    return ret

def cointegration(y, x, period = 0):
    adf = DickeyFuller()
    n = len(y)
    if (period > 0):
        pos = n-period
        y = y.iloc[pos:]
        x = x.iloc[pos:]
    else:
        period = n
        
    model = model_ols(y, x)
    adf.check(model.resid)
    return [adf.is_stationary, adf.p_value, adf.perc_stat, model.params[1], period]

def apply_halflife(data, pairs):
    pairs['HalfLife'] = 0
    
    for i, row in pairs.iterrows():
        y = data[row['Dependente']]
        x = data[row['Independente']]
        
        value = check_halflife(y, x)
        pairs['HalfLife'].iloc[i]=value

def check_halflife(y, x):
    halflile = HalfLife()
    model = model_ols(y, x)
    return halflile.check(model.resid)

def apply_hurst(data, pairs):
    pairs['Hurst'] = 0
    
    for i, row in pairs.iterrows():
        y = data[row['Dependente']]
        x = data[row['Independente']]
        
        value = check_hurst(y, x)
        pairs['Hurst'].iloc[i]= value

def check_hurst(y, x):
    hurst = HurstExponent()
    model = model_ols(y, x)
    return hurst.check(model.resid.values)

# 0.9 para mais ou para menos indica uma correlação muito forte.
# 0.7 a 0.9 positivo ou negativo indica uma correlação forte.percorre
# 0.5 a 0.7 positivo ou negativo indica uma correlação moderada.
# 0.3 a 0.5 positivo ou negativo indica uma correlação fraca.
# 0 a 0.3 positivo ou negativo indica uma correlação desprezível.'''
def corr_pearson(y, x):
    y_avg, x_avg = np.average(y), np.average(x)
    y_stdev, x_stdev = np.std(y), np.std(x)
    n = len(y)
    denominator = y_stdev * x_stdev * n
    numerator = np.sum(np.multiply(y-y_avg, x-x_avg))
    p_coef = numerator/denominator
    return p_coef

def apply_corr(data, pairs):
    pairs['Corr'] = 0
    
    for i, row in pairs.iterrows():
        y = data[row['Dependente']]
        x = data[row['Independente']]
        
        corr = corr_pearson(return_varlog(y), return_varlog(x))
        pairs['Corr'].iloc[i] = corr

def signal(y, x):
    model = model_ols(y, x)
    std = statistics.stdev(model.resid)
    resi_curr = model.resid.iloc[-1]
    zscore_up = 2*std
    zscore_down = -2*std
    zcurrent = 0
    desc = ''
    
    # >0; resíduo acima da linha 0
    if(resi_curr > 0):
        desc = 'Short/Long'
        zcurrent = zscore_up
    else:
        desc = 'Long/Short'
        zcurrent = zscore_down
    
    percent = (abs(resi_curr)/abs(zcurrent))
    #1-descr
    #2-resíduo atual
    #3-percent distância da linha 0, quanto maior, melhor
    return [desc, resi_curr, percent]

def apply_signal(data, pairs):
    pairs['Signal'] = 0
    pairs['SignalStr'] = ''    
    
    for i, row in pairs.iterrows():
        y = data[row['Dependente']]
        x = data[row['Independente']]

        sig = signal(y, x)
        pairs['Signal'].iloc[i] = sig[2]
        pairs['SignalStr'].iloc[i] = sig[0]        

def check_periods(data, y_symbol, x_symbol, period):
    if (type(period) is int):
        return check_oneperiod(data, y_symbol, x_symbol, period)
    if (type(period) is list):
        rows=[]
        for p in period:
            res = check_oneperiod(data, y_symbol, x_symbol, p)
            rows.append([res[0], res[1]])
        return rows

def check_oneperiod(data, y_symbol, x_symbol, period):
    y = data[y_symbol]
    x = data[x_symbol]
    
    if(period > 0):
        pos = data.shape[0]-period
        y = y.iloc[pos:]
        x = x.iloc[pos:]
    
    adf = DickeyFuller()
    model = model_ols(y, x)
    adf.check(model.resid)
    beta = model.params[1]

    return [adf.p_value, adf.is_stationary];

def show(data, y_symbol, x_symbol, period=0):
    y = data[y_symbol]
    x = data[x_symbol]
    
    if(period > 0):
        pos = data.shape[0]-period
        y = y.iloc[pos:]
        x = x.iloc[pos:]
    
    model= model_ols(y, x)
    std = statistics.stdev(model.resid)
    entry_threshold = 2 # entrada em 2 desvio padrão

    #plt.figure(figsize=(15,6))
    plt.figure(figsize=(10,5))
    plt.plot(model.resid)
    plt.ylabel('Residual')
    plt.title(y_symbol + ' / ' + x_symbol)

    plt.axhline(0, color='black',label='mean',linestyle='--') # Add the mean of residual
    plt.axhline(entry_threshold*std, color='green', linestyle='--',label='trade')
    plt.axhline(-entry_threshold*std, color='green', linestyle='--')

    plt.legend()
    plt.show()

## Read CSV

In [42]:
path_data_cart = 'datasets/data_cart.csv'
path_data_full = 'datasets/data.csv'
path_data_alpha = 'datasets/data_alpha.csv'

In [43]:
df = pd.read_csv(path_data_cart, index_col=0)#[['ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3']]
data = df[df.columns.difference(['Data', 'Date'])]
data.shape

(300, 71)

In [40]:
pairs = find_cointegrated_pairs(data)
pairs.head(3)

,Period,Dependente,Independente,Dickey-Fuller,ADF,Beta
0,250,ABEV3,BBDC3,0.028365,95,0.439228
1,250,ABEV3,BBDC4,0.018269,95,0.407722
2,250,ABEV3,BBSE3,0.045900,95,0.601563


In [44]:
apply_halflife(data, pairs)
print('Half-Life applied successfully')

Half-Life applied successfully


In [45]:
apply_hurst(data, pairs)
print('Hurst applied successfully')

Hurst applied successfully


In [63]:
apply_corr(data, pairs)
print('Correlation applied successfully')

Correlation applied successfully


In [51]:
apply_signal(data, pairs)
print('Signal applied successfully')

Signal applied successfully


In [64]:
pairs.head(3)

,Period,Dependente,Independente,Dickey-Fuller,ADF,Beta,HalfLife,Hurst,Corr,Signal,SignalStr
0,250,ABEV3,BBDC3,0.028365,95,0.439228,8,0.383536,0.545728,0.345194,Short/Long
1,250,ABEV3,BBDC4,0.018269,95,0.407722,9,0.372016,0.539330,0.372590,Short/Long
2,250,ABEV3,BBSE3,0.045900,95,0.601563,19,0.300201,0.430605,0.188973,Long/Short


In [71]:
pairs.to_csv('datasets/cointegrated_pairs.csv', index=False)
print('Criado com sucesso!!!')

Criado com sucesso!!!


# Analysis

In [72]:
pairs = pd.read_csv('datasets/cointegrated_pairs.csv')
pairs.shape

(274, 11)

In [74]:
df_hurst = pairs.loc[(pairs['Hurst'] > 0) & (pairs['Hurst'] < 0.4)]
df_hurst

,Period,Dependente,Independente,Dickey-Fuller,ADF,Beta,HalfLife,Hurst,Corr,Signal,SignalStr
0,250,ABEV3,BBDC3,0.028365,95,0.439228,8,0.383536,0.920057,0.345194,Short/Long
1,250,ABEV3,BBDC4,0.018269,95,0.407722,9,0.372016,0.906131,0.372590,Short/Long
2,250,ABEV3,BBSE3,0.045900,95,0.601563,19,0.300201,0.808525,0.188973,Long/Short
5,250,ABEV3,CVCB3,0.025185,95,0.171163,12,0.370997,0.915166,0.105496,Long/Short
9,250,ABEV3,HGTX3,0.007722,99,0.350596,7,0.318604,0.958284,0.792498,Short/Long
...,...,...,...,...,...,...,...,...,...,...,...
260,250,SBSP3,TAEE11,0.003565,99,4.212692,8,0.308825,0.877747,0.071651,Short/Long
263,250,SULA11,TAEE11,0.001978,99,5.012405,6,0.384851,0.912135,0.594746,Long/Short
265,250,SULA11,VIVT4,0.007186,99,1.822508,11,0.348087,0.840291,0.493076,Short/Long
270,250,TAEE11,VIVT4,0.039851,95,0.321738,9,0.386550,0.841041,0.988618,Short/Long


In [70]:
corr = df_hurst.loc[(df_hurst['Corr'] >= 0.70)]
corr

,Period,Dependente,Independente,Dickey-Fuller,ADF,Beta,HalfLife,Hurst,Corr,Signal,SignalStr
0,250,ABEV3,BBDC3,0.028365,95,0.439228,8,0.383536,0.920057,0.345194,Short/Long
1,250,ABEV3,BBDC4,0.018269,95,0.407722,9,0.372016,0.906131,0.372590,Short/Long
2,250,ABEV3,BBSE3,0.045900,95,0.601563,19,0.300201,0.808525,0.188973,Long/Short
5,250,ABEV3,CVCB3,0.025185,95,0.171163,12,0.370997,0.915166,0.105496,Long/Short
9,250,ABEV3,HGTX3,0.007722,99,0.350596,7,0.318604,0.958284,0.792498,Short/Long
...,...,...,...,...,...,...,...,...,...,...,...
260,250,SBSP3,TAEE11,0.003565,99,4.212692,8,0.308825,0.877747,0.071651,Short/Long
263,250,SULA11,TAEE11,0.001978,99,5.012405,6,0.384851,0.912135,0.594746,Long/Short
265,250,SULA11,VIVT4,0.007186,99,1.822508,11,0.348087,0.840291,0.493076,Short/Long
270,250,TAEE11,VIVT4,0.039851,95,0.321738,9,0.386550,0.841041,0.988618,Short/Long


#### Pares sinalizando entrada

In [57]:
signal = corr.loc[(df_hurst['Signal'] >= 0.95)].copy()
signal.reset_index(drop=True, inplace=True)
apply_periods(data, signal)

In [58]:
# somente pares cointegrados em no mínimo 3 períodos
signal.loc[(signal['Period'] > 2)].sort_values(by=['HalfLife'], ascending=True)

,Period,Dependente,Independente,Dickey-Fuller,ADF,Beta,HalfLife,Hurst,Corr,Signal,SignalStr,PeriodStr
1,8,BRML3,LREN3,0.014814,95,0.412077,5,0.239897,0.967598,0.974424,Long/Short,"100,140,160,180,200,220,240,250"
11,9,LREN3,SBSP3,0.007779,99,0.859622,8,0.198421,0.897282,1.076355,Long/Short,"100,120,140,160,180,200,220,240,250"
14,7,TOTS3,VIVT4,0.000800,99,3.417144,8,0.302502,0.811557,1.294074,Long/Short,"140,160,180,200,220,240,250"
3,9,CYRE3,MULT3,0.005319,99,1.252504,9,0.283458,0.956891,0.994709,Short/Long,"100,120,140,160,180,200,220,240,250"
4,7,CYRE3,SBSP3,0.004311,99,0.739383,9,0.260135,0.932530,1.076850,Long/Short,"140,160,180,200,220,240,250"
10,8,HYPE3,VIVT4,0.008386,99,0.591906,9,0.348168,0.783047,0.982878,Short/Long,"120,140,160,180,200,220,240,250"
5,7,ECOR3,VIVT4,0.006333,99,0.560235,10,0.266502,0.878609,1.002586,Short/Long,"140,160,180,200,220,240,250"
7,7,ENGI11,MULT3,0.014519,95,0.973896,11,0.365437,0.879206,1.033829,Short/Long,"100,120,180,200,220,240,250"
0,3,BPAC11,ENBR3,0.022419,95,5.979289,12,0.329108,0.876672,0.975123,Short/Long,"220,240,250"
2,3,CPFE3,MULT3,0.045652,95,0.559100,12,0.355238,0.856351,1.130766,Short/Long,"220,240,250"
